In [1]:
# Reimportar librerías después del reinicio del estado
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler,LabelEncoder
from sklearn.model_selection import train_test_split


In [2]:
df= pd.read_csv("/Users/isaromobru/Desktop/FakeNews 10.57.24/proyecto_machine_learning_Fake_News/data/processed/archivo.csv")

In [3]:
le = LabelEncoder()
df['subject_encoded'] = le.fit_transform(df['subject_grouped'])


In [4]:
from scipy.sparse import csr_matrix, hstack
# Eliminar Na
df['text'] = df['text'].fillna("")

# Vectoriza la columna 'text'
vectorizer_text = TfidfVectorizer(max_features=5000)
X_text = vectorizer_text.fit_transform(df['text'])

# Vectoriza la columna 'title'
vectorizer_title = TfidfVectorizer(max_features=1000)
X_title = vectorizer_title.fit_transform(df['title'])

# Convierte la columna 'subject_encoded' a una matriz dispersa
subject_feature = csr_matrix(df['subject_encoded'].values.reshape(-1, 1))

# Combina las tres fuentes de información en una única matriz de características
X = hstack([subject_feature, X_title, X_text])

# Define la variable objetivo; en este ejemplo, se asume que la columna "label" es la etiqueta a predecir
y = df['label']

# Divide los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:

# Definir los modelos y sus hiperparámetros a optimizar
param_grid = {
    'RandomForest': {
        'model': RandomForestClassifier(),
        'params': {
            'n_estimators': [50, 100, 200],
            'max_depth': [None, 10, 20],
            'min_samples_split': [2, 5, 10],
        }
    }
}

In [6]:
# Realizar la búsqueda en malla para cada modelo
best_models = {}
for name, model_info in param_grid.items():
    print(f"\n🔍 Optimizando {name}...")
    grid_search = GridSearchCV(model_info['model'], model_info['params'], cv=3, scoring='accuracy', n_jobs=-1)
    grid_search.fit(X_train, y_train)  # Usamos la representación TF-IDF de los datos
    
    best_models[name] = {
        'best_score': grid_search.best_score_,
        'best_params': grid_search.best_params_
    }
    print(f"✅ Mejor precisión para {name}: {grid_search.best_score_:.4f}")
    print(f"📌 Mejores hiperparámetros: {grid_search.best_params_}")



🔍 Optimizando RandomForest...
✅ Mejor precisión para RandomForest: 0.9981
📌 Mejores hiperparámetros: {'max_depth': None, 'min_samples_split': 5, 'n_estimators': 200}


In [9]:


# Definir los modelos y sus hiperparámetros a optimizar
param_gridxg = {
    
    'XGBoost': {
        'model': XGBClassifier( eval_metric='logloss'),
        'params': {
            'n_estimators': [50, 100, 200],
            'max_depth': [3, 6, 10],
            'learning_rate': [0.01, 0.1, 0.2],
        }
 
    }
}


In [10]:
# Realizar la búsqueda en malla para cada modelo
best_models1 = {}
for name, model_info in param_gridxg.items():
    print(f"\n🔍 Optimizando {name}...")
    grid_search = GridSearchCV(model_info['model'], model_info['params'], cv=3, scoring='accuracy', n_jobs=-1)
    grid_search.fit(X_train, y_train)  # Usamos la representación TF-IDF de los datos
    grid_search.fit(X_train, y_train)  # Usamos la representación TF-IDF de los datos
    
    best_models[name] = {
        'best_score': grid_search.best_score_,
        'best_params': grid_search.best_params_
    }
    print(f"✅ Mejor precisión para {name}: {grid_search.best_score_:.4f}")
    print(f"📌 Mejores hiperparámetros: {grid_search.best_params_}")



🔍 Optimizando XGBoost...


KeyboardInterrupt: 

In [ ]:


# Definir los modelos sus hiperparámetros a optimizar
param_grid = {
    'RandomForest': {
        'model': RandomForestClassifier(),
        'params': {
            'n_estimators': [50, 100, 200],
            'max_depth': [None, 10, 20],
            'min_samples_split': [2, 5, 10],
        }
    },
    'XGBoost': {
        'model': XGBClassifier(use_label_encoder=False, eval_metric='logloss'),
        'params': {
            'n_estimators': [50, 100, 200],
            'max_depth': [3, 6, 10],
            'learning_rate': [0.01, 0.1, 0.2],
        }
 
    }
}

# Realizar la búsqueda en malla para cada modelo
best_models = {}
for name, model_info in param_grid.items():
    print(f"\n🔍 Optimizando {name}...")
    grid_search = GridSearchCV(model_info['model'], model_info['params'], cv=3, scoring='accuracy', n_jobs=-1)
    grid_search.fit(X_tfidf, y)  # Usamos la representación TF-IDF de los datos
    
    best_models[name] = {
        'best_score': grid_search.best_score_,
        'best_params': grid_search.best_params_
    }
    print(f"✅ Mejor precisión para {name}: {grid_search.best_score_:.4f}")
    print(f"📌 Mejores hiperparámetros: {grid_search.best_params_}")

# Convertir resultados a DataFrame y mostrar
df_best_models = pd.DataFrame(best_models).T
#import ace_tools as tools
#tools.display_dataframe_to_user(name="Mejores Modelos con GridSearchCV", dataframe=df_best_models)


In [11]:
import numpy as np
import pandas as pd
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from scipy.stats import randint, uniform
from sklearn.metrics import accuracy_score


# -------------------------------------
# 1. Definir modelos y espacios de búsqueda
# -------------------------------------
models_params = {
    "RandomForest": {
        "model": RandomForestClassifier(random_state=42),
        "params": {
            "n_estimators": randint(50, 200),
            "max_depth": randint(5, 50),
            "min_samples_split": randint(2, 10),
            "min_samples_leaf": randint(1, 5)
        }
    },
    "GradientBoosting": {
        "model": GradientBoostingClassifier(random_state=42),
        "params": {
            "n_estimators": randint(50, 200),
            "learning_rate": uniform(0.01, 0.3),
            "max_depth": randint(3, 10),
            "min_samples_split": randint(2, 10),
            "min_samples_leaf": randint(1, 5)
        }
    },
    "XGBoost": {
        "model": XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42),
        "params": {
            "n_estimators": randint(50, 200),
            "learning_rate": uniform(0.01, 0.3),
            "max_depth": randint(3, 10),
            "min_child_weight": randint(1, 6),
            "gamma": uniform(0, 0.3),
            "subsample": uniform(0.5, 0.5),
            "colsample_bytree": uniform(0.5, 0.5)
        }
    }
}

# -------------------------------------
# 2. Aplicar Random Search para cada modelo
# -------------------------------------
best_models = {}
results = []

for model_name, mp in models_params.items():
    print(f"\n🔍 Optimizando {model_name} con RandomizedSearchCV...\n")
    
    # Configurar RandomizedSearchCV
    search = RandomizedSearchCV(
        mp["model"], mp["params"], 
        n_iter=20, cv=3, scoring="accuracy", 
        n_jobs=-1, random_state=42, verbose=1
    )
    
    # Entrenar la búsqueda
    search.fit(X_train, y_train)
    
    # Guardar mejor modelo y resultado
    best_models[model_name] = search.best_estimator_
    
    # Evaluar en conjunto de prueba
    y_pred = search.best_estimator_.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    
    # Guardar resultados
    results.append({
        "Modelo": model_name,
        "Mejores Hiperparámetros": search.best_params_,
        "Precisión en Test": accuracy
    })


🔍 Optimizando RandomForest con RandomizedSearchCV...

Fitting 3 folds for each of 20 candidates, totalling 60 fits

🔍 Optimizando GradientBoosting con RandomizedSearchCV...

Fitting 3 folds for each of 20 candidates, totalling 60 fits
